In [49]:
class signature_oracle:
    def __init__(self, q):
        self.q = q
    def Sign(self, m, sk):
        return (pow(m, sk, self.q))
    
    def Verify(self, m, sk, s):
        return (s==self.Sign(m, sk))

In [50]:
def Com(w, r):
    return (w, r)

In [51]:
# has secret x
class Prover:
    def __init__(self, g, q, N):
        self.honest = True
        self.g = g
        self.q = q
        self.k = randint(1, q)
        self.x = -99999
        self.w = -99999
        self.r = -99999
    
    def Commit(self, v):
        self.w = v^self.x
        self.r = randint(1, self.q)
        return Com(self.w, self.r)
    
    def Verify(self, v, m, a, b):
        t = (pow(m, a, self.q)*pow(self.g, b, self.q)) % self.q
        return (v == t)

In [52]:
class Verifier:
    def __init__(self, g, q, N):
        self.g = g
        self.q = q
        self.a = randint(1,self.q)
        self.b = randint(1,self.q)
        
    def Generate(self, m):
        v = (pow(m, self.a, self.q)*pow(self.g, self.b, self.q)) % self.q
        return v
    
    def Verify(self, s, yA, c, w, r):
        t = (pow(s, self.a, self.q)*pow(yA, self.b, self.q)) % self.q
        v1 = (w == t)
        v2 = (c == Com(w, r))
        return (v1 and v2)

In [53]:
import time
class DVP_Protocol:
    def __init__(self, bits, seed):
        start = 0
        end = 0
        
        set_random_seed(seed)
        current_randstate().set_seed_gp()
        
        N = 2^bits
        q = random_prime(N)
        Zq = Integers(q)
        g = Zq(2)
        
        # message
        m = randint(1, q)
        # secret key
        xA = randint(1, q)
        
        prover = Prover(g, q, N)
        verifier = Verifier(g, q, N)
        
        if prover.honest:
            prover.x = xA
        else:
            prover.x = randint(1, q)
        
        yP = g^prover.x
        S = signature_oracle(q)
        s = S.Sign(m, prover.x)
        
        #print("Generate part......")
        start = time.time()
        v = verifier.Generate(m)
        end = time.time()
        self.generate_time = end-start
        
        #print("Commit part......")
        start = time.time()
        c = prover.Commit(v)
        end = time.time()
        self.commit_time = end-start
        
        #print("Prover verify part......")
        start = time.time()
        b1 = prover.Verify(v, m, verifier.a, verifier.b)
        end = time.time()
        self.pv_time = end-start
        #print(b1)
        
        #print("Verifier verify part......")
        start = time.time()
        b2 = verifier.Verify(s, yP, c, prover.w, prover.r)
        end = time.time()
        self.vv_time = end-start
        #print(b2)
        
        self.result = (b1 and b2)

In [54]:
bits = 2048
ctr = 0
iteration = 20
total_generate_time = 0
total_commit_time = 0
total_pv_time = 0
total_vv_time = 0

while ctr < iteration:
    seed = time.time_ns()
    print("iteration : %d / %d" % (ctr+1, iteration), end="\r")

    dvp = DVP_Protocol(bits, seed)
    if dvp.result:
        ctr += 1
        total_generate_time += dvp.generate_time*10^3
        total_commit_time += dvp.commit_time*10^3
        total_pv_time += dvp.pv_time*10^3
        total_vv_time += dvp.vv_time*10^3
    else:
        pass

print("\n-----------------------------")
print("Total generate time / Average generate time :               %.2fms / %.2fms" % (total_generate_time, total_generate_time/iteration))
print("Total commit time / Average commit time :                   %.2fms / %.2fms" % (total_commit_time, total_commit_time/iteration))
print("Total prover verify time / Average prover verify time :     %.2fms / %.2fms" % (total_pv_time, total_pv_time/iteration))
print("Total verifier verify time / Average verifier verify time : %.2fms / %.2fms" % (total_vv_time, total_vv_time/iteration))


iteration : 20 / 20
-----------------------------
Total generate time / Average generate time :               157.90ms / 7.90ms
Total commit time / Average commit time :                   78.62ms / 3.93ms
Total prover verify time / Average prover verify time :     157.58ms / 7.88ms
Total verifier verify time / Average verifier verify time : 156.46ms / 7.82ms
